# MCP and AgentCore Authentication with Okta 

This notebook provides a comprehensive guide to deploying and testing an MCP (Model Context Protocol) Agent on AWS Bedrock AgentCore with Okta OAuth authentication.

## Overview

The MCP Agent integrates external data sources through the Model Context Protocol, enabling AI agents to access real-time information from AWS services like Pricing and Documentation APIs. This setup demonstrates secure authentication flows using Okta OAuth 2.0 with Bedrock Agent Core.

### Architecture Overview

```
┌────────┐    1. Credentials    ┌───────┐    2. JWT Token    ┌────────┐
│ Client │ ──────────────────► │ Okta  │ ─────────────────► │ Client │
└────────┘                     └───────┘                    └───┬────┘
                                                                │
                                                                │ 3. Bearer JWT
                                                                ▼
┌────────┐    6. Response    ┌──────────────┐    4. Invoke    ┌───────────┐
│ Client │ ◄──────────────── │ Bedrock      │ ──────────────► │ MCP Agent │
└────────┘                  │ Agent Core   │ ◄──────────────── └───┬───────┘
                            └──────────────┘    5. Response        │
                                                                   │ MCP stdio
                                                                   ▼
                                                            ┌─────────────┐
                                                            │ MCP Servers │
                                                            │ (Pricing,   │
                                                            │  Docs)      │
                                                            └─────────────┘
```

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
|                                                        |
| Agent type          | MCP-enabled                                                                      |
| Agentic Framework   | Strands Agents with MCP                                                         |
| LLM model           | Anthropic Claude                                              |
| Tutorial components | MCP Servers, Bedrock AgentCore, Okta OAuth                                     |                                        |
| Example complexity  | Intermediate                                                                     |
| Inbound Auth        | Okta OAuth 2.0                                                                  |
| SDK used            | bedrock-agentcore, strands, mcp                                                 |

## Prerequisites

- AWS Account with Bedrock Agent Core access
- Okta Developer Account
- Python 3.9+
- AWS CLI configured
- Docker (for containerized deployment)


## Step 1: Environment Setup

First, let's set up the development environment and install required dependencies.

In [ ]:
# Create and activate virtual environment
!python -m venv .venv
!source .venv/bin/activate

For this code to run, the Strands Agents modules need to be installed in the Python environment.

Add the Strands Agents modules, AgentCore SDK, and AgentCore starter toolkit to the dependency file and save it as **requirements.txt**:

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit

In [ ]:
# Install required packages
!pip install -r requirements.txt

In [ ]:
# Verify installations
try:
    import bedrock_agentcore
    import strands
    print("✅ All packages installed successfully")
    print("✅ bedrock-agentcore: imported")
    print("✅ strands: imported")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure all packages are installed correctly")

## Step 2: Okta Configuration

Configure Okta OAuth application for authentication.

### 2.1 Create Okta Application

1. Log into your Okta Developer Console
2. Navigate to **Applications** → **Create App Integration**
3. Select **API Services** (Machine-to-Machine)
4. Configure the application:
   - **App Name**: `MCP Agent Core`
   - **Grant Types**: Client Credentials
   - **Scopes**: `agentcore`

### 2.2 Configure Environment Variables

In [ ]:
import os

# Okta Configuration - Replace with your values
os.environ['OKTA_CLIENT_ID'] = 'YOUR CLIENT ID VALUE'
os.environ['OKTA_CLIENT_SECRET'] = 'YOUR CLIENT SECRET VALUE'
os.environ['OKTA_AUDIENCE'] = 'testagentcore'
os.environ['OKTA_TOKEN_URL'] = 'https://your.okta.com/oauth2/default/v1/token'
os.environ['OKTA_DISCOVERY_URL'] = 'https://your.okta.com/oauth2/default/.well-known/openid-configuration'

print("✅ Okta environment variables configured")

## Step 3: MCP Agent Implementation

Create the MCP Agent that integrates with AWS Pricing and Documentation APIs.

In [ ]:
%%writefile my_mcp_agent.py
#!/usr/bin/env python3
"""
AWS Solutions Architect - MCP Agent with Pricing and Documentation Integration
"""

from bedrock_agentcore import BedrockAgentCoreApp
from mcp import StdioServerParameters, stdio_client
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient

# MCP Servers - Cost Explorer and Documentation
aws_pricing_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

aws_docs_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    temperature=0.7,
)

SYSTEM_PROMPT = """
You are an expert AWS Solutions Architect specializing in database migrations.
Create comprehensive migration documentation following AWS Well-Architected Framework.
Use AWS documentation for best practices and pricing analysis for TCO calculations.
"""

app = BedrockAgentCoreApp()

@app.entrypoint
def invoke(payload):
    """Handler for agent invocation"""
    prompt = payload.get("prompt", "Please provide a prompt for the migration guide")
    
    print("Initializing MCP servers...")
    with aws_pricing_client, aws_docs_client:
        print("Getting available tools...")
        all_tools = (
            aws_pricing_client.list_tools_sync() +
            aws_docs_client.list_tools_sync()
        )
        print(f"Found {len(all_tools)} tools")
        
        agent = Agent(tools=all_tools, model=bedrock_model, system_prompt=SYSTEM_PROMPT)

        print("Creating migration guide...")
        response = agent(prompt)
        
        return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## Step 4: Bedrock Agent Core Configuration using Okta as Authorizer

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

# Use environment variables or defaults (don't use input() in notebooks)
discovery_url = os.getenv('OKTA_DISCOVERY_URL')
client_id = os.getenv('OKTA_CLIENT_ID')
audience = os.getenv('OKTA_AUDIENCE')

print(f"Discovery URL: {discovery_url}")
print(f"Client ID: {client_id[:4]}****{client_id[-4:]}")  # Masked
print(f"Audience: {audience}")

agentcore_runtime = Runtime()

# Try with OAuth configuration
try:
    response = agentcore_runtime.configure(
        entrypoint="my_mcp_agent.py",  # Correct filename
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="mcp_agent_okta_authfri",  # Consistent naming
        authorizer_configuration={
            "customJWTAuthorizer": {
                "discoveryUrl": discovery_url,
                "allowedClients": [client_id],
                "allowedAudience": [audience]  # Must be a list, not string
            }
        }
    )
    print("✅ OAuth configuration successful")
except Exception as e:
    print(f"❌ OAuth configuration failed: {e}")
    print("Trying without OAuth configuration...")
    
    # Fallback without OAuth
    response = agentcore_runtime.configure(
        entrypoint="my_mcp_agent.py",
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="mcp_agent_okta_auth"
    )
    print("✅ Basic configuration successful (OAuth can be added later)")

response


### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Checking for the AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for its deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Save Agent ARN for Testing

Extract and save the Agent ARN for use in testing

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using region: {region}")

In [ ]:
import os

# Extract ARN from launch_result (available immediately after launch)
if hasattr(launch_result, 'agent_arn') and launch_result.agent_arn:
    agent_arn = launch_result.agent_arn
    os.environ['AGENT_ARN'] = agent_arn
    print(f"📝 Agent ARN: {agent_arn}")
    print(f"📝 Agent ID: {launch_result.agent_id}")
    print(f"📝 ECR URI: {launch_result.ecr_uri}")
else:
    print("⚠️  Could not extract Agent ARN from launch result")
    print("Launch result:", launch_result)

# Also check final deployment status
if status == 'READY':
    print("✅ Agent deployed successfully and ready for testing!")
elif status in ['CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
    print(f"❌ Agent deployment failed with status: {status}")
    print("Check the status response for error details")
else:
    print(f"⚠️  Unexpected status: {status}")

## Step 6: Create Test Client

Now lets create a test client to validate the Okta OAuth flow and agent invocation.

In [ ]:
import requests
import json
import time
import urllib.parse
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
client_id = os.getenv('OKTA_CLIENT_ID')
client_secret = os.getenv('OKTA_CLIENT_SECRET')
audience = os.getenv('OKTA_AUDIENCE')
token_url = os.getenv('OKTA_TOKEN_URL')
AGENT_ARN = os.getenv('AGENT_ARN')

print(f"Client ID: {client_id}")
print(f"Token URL: {token_url}")
print(f"Agent ARN: {AGENT_ARN}")
print(f"token URL: {token_url}")

Function to generate oauth request from Okta 

In [ ]:
def get_oauth_token():
    """Get OAuth token from Okta"""
    data = {
        'grant_type': 'client_credentials',
        'scope': 'agentcore'
    }
    
    logger.info("🔐 Getting OAuth token...")
    
    response = requests.post(
        token_url,
        data=data,
        auth=(client_id, client_secret)
    )
    
    response.raise_for_status()
    token_data = response.json()
    logger.info("✅ OAuth token obtained")
    return token_data['access_token']

Function to invoke Bedrockagentcore Runtime

In [ ]:
def invoke_agent(access_token, query):
    """Invoke MCP agent"""
    escaped_agent_arn = urllib.parse.quote(AGENT_ARN, safe='')
    url = f"https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    session_id = f'test-session-{int(time.time())}-xxxxxxxxxx'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
    }
    
    payload = {'prompt': query}
    
    logger.info(f"🚀 Invoking agent with query: {query}")
    
    response = requests.post(url, headers=headers, json=payload, timeout=300)
    response.raise_for_status()
    
    result = response.json()
    logger.info("✅ Agent response received")
    return result

In [ ]:
# Get OAuth token
access_token = get_oauth_token()
print(f"Token obtained from OKTA: {access_token[:20]}...")

In [ ]:
# Test 1: Simple query to test the agent
print("=" * 50)
print("TEST 1: Simple Query")
print("=" * 50)

result1 = invoke_agent(access_token, "Hello, can you help me?")
print(json.dumps(result1, indent=2))

In [ ]:
# Test 2: Query which will use AWS MCP doc server to answer
print("=" * 50)
print("TEST 2: MCP Query")
print("=" * 50)

result2 = invoke_agent(access_token, "What is the best practices for RDS Aurora MySQL DR?")
print(json.dumps(result2, indent=2))

In [ ]:
#!/usr/bin/env python3
import requests
import json
import time
import urllib.parse
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configure verbose logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Enable urllib3 debug logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.DEBUG)

# Configuration
client_id = os.getenv('OKTA_CLIENT_ID')
client_secret = os.getenv('OKTA_CLIENT_SECRET')
audience = os.getenv('OKTA_AUDIENCE')
token_url = os.getenv('OKTA_TOKEN_URL')
AGENT_ARN = os.getenv('AGENT_ARN')

def create_session_with_timeout():
    """Create requests session with timeout and retry configuration"""
    session = requests.Session()
    
    # Configure retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def get_oauth_token():
    """Get OAuth token from Okta"""
    session = create_session_with_timeout()
        
    data = {
        'grant_type': 'client_credentials',
        'scope': 'agentcore'
    }
    
    try:
        logger.info("🔐 Getting OAuth token...")
        logger.debug(f"Token URL: {token_url}")
        logger.debug(f"Client ID: {client_id}")
        logger.debug(f"Request data: {data}")
        
        response = session.post(
            token_url,
            data=data,
            auth=(client_id, client_secret),
            timeout=30
        )
        
        logger.debug(f"OAuth response status: {response.status_code}")
        logger.debug(f"OAuth response headers: {dict(response.headers)}")
        logger.debug(f"OAuth response body: {response.text}")
        
        response.raise_for_status()
        
        token_data = response.json()
        logger.info("✅ OAuth token obtained")
        return token_data['access_token']
        
    except requests.exceptions.Timeout:
        logger.error("❌ OAuth request timed out")
        return None
    except Exception as e:
        logger.error(f"❌ OAuth error: {e}")
        return None

def invoke_agent_with_timeout(access_token, query="What is the pricing for RDS MYSQL r5g t3.micro instances?"):
    """Invoke agent with timeout handling"""
    session = create_session_with_timeout()
    
    escaped_agent_arn = urllib.parse.quote(AGENT_ARN, safe='')
    url = f"https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    session_id = f'test-session-{int(time.time())}-xxxxxxxxxx'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
    }
    
    payload = {
        'prompt': query
    }
    
    try:
        logger.info("🚀 Invoking MCP Agent with 300 timeout...")
        logger.info(f"📝 Query: {query}")
        logger.debug(f"🌐 URL: {url}")
        logger.debug(f"📋 Headers: {headers}")
        logger.debug(f"📦 Payload: {json.dumps(payload, indent=2)}")
        logger.debug(f"🆔 Session ID: {session_id}")
        logger.debug(f"🔑 Token (first 20 chars): {access_token[:20]}...")
        
        start_time = time.time()
        
        logger.debug("📤 Sending POST request...")
        response = session.post(
            url,
            headers=headers,
            json=payload,
            timeout=300
        )
        
        elapsed = time.time() - start_time
        logger.info(f"⏱️  Request completed in {elapsed:.2f} seconds")
        
        logger.debug(f"📊 Response Status: {response.status_code}")
        logger.debug(f"📋 Response Headers: {dict(response.headers)}")
        logger.debug(f"📦 Response Body: {response.text}")
        
        if response.status_code == 200:
            result = response.json()
            logger.info("✅ Agent Response:")
            print(json.dumps(result, indent=2))
        else:
            logger.error(f"❌ Error Response ({response.status_code}): {response.text}")
            
    except requests.exceptions.Timeout:
        logger.error("❌ Agent invocation timed out after 300 seconds")
        logger.error("💡 This suggests the agent may be stuck or taking too long to process")
        
    except requests.exceptions.ConnectionError as e:
        logger.error(f"❌ Connection error: {e}")
        
    except Exception as e:
        logger.error(f"❌ Unexpected error: {e}")
        logger.exception("Full exception details:")

def main():
    logger.info("🔍 Testing MCP Agent with verbose logging...")
    
    # Get OAuth token
    access_token = get_oauth_token()
    if not access_token:
        logger.error("❌ Failed to get OAuth token")
        return
    
    # Test with a simple query first
    logger.info("\n" + "="*50)
    logger.info("TEST 1: Simple Query")
    logger.info("="*50)
    invoke_agent_with_timeout(access_token, "Hello, can you help me?")
    
    logger.info("\n" + "="*50)
    logger.info("TEST 2: MCP Query")
    logger.info("="*50)
    
    # Test with archiecture query
    invoke_agent_with_timeout(access_token, "What is the best practices for RDS Aurora Mysql DR?")

if __name__ == "__main__":
    main()


## Troubleshooting

### Common Issues and Solutions

#### 1. Authentication Errors
```
Error: Failed to get token: 401
```
**Solution**: Verify Okta credentials and configuration
- Check `OKTA_CLIENT_ID` and `OKTA_CLIENT_SECRET`
- Ensure the Okta application has correct grant types
- Verify the token URL format

#### 2. Agent ARN Not Found
```
Error: AGENT_ARN environment variable is required
```
**Solution**: Set the correct Agent ARN from deployment output

#### 3. MCP Server Connection Issues
```
Error: Failed to connect to MCP server
```
**Solution**: 
- Ensure `uvx` is installed in the runtime environment
- Check network connectivity for MCP server downloads
- Verify MCP server package names

#### 4. Permission Errors
```
Error: 403 Forbidden
```
**Solution**: 
- Check Bedrock Agent Core IAM permissions
- Verify OAuth audience and client configuration
- Ensure the JWT token has correct scopes

### Debug Mode

Enable verbose logging for detailed troubleshooting:

In [ ]:
import logging

# Enable debug logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger("urllib3.connectionpool").setLevel(logging.DEBUG)

print("🔍 Debug mode enabled - run your test again for detailed logs")

## Architecture Deep Dive

### Authentication Flow Details

The complete authentication and invocation flow consists of these layers:

1. **OAuth 2.0 Client Credentials Flow**
   - Client authenticates with Okta using client credentials
   - Receives JWT access token with `agentcore` scope
   - Token valid for 1 hour (configurable)

2. **Bedrock Agent Core Authentication**
   - Client presents JWT as Bearer token
   - Bedrock validates token against Okta OIDC endpoint
   - Session tracking via unique session IDs

3. **MCP Protocol Communication**
   - Agent communicates with MCP servers via stdio
   - No additional authentication required for MCP layer
   - MCP servers handle AWS API authentication via IAM

### Security Considerations

- **Transport Security**: All communications use HTTPS/TLS
- **Token Validation**: JWT signatures verified against Okta public keys
- **Scope Limitation**: Tokens limited to `agentcore` scope
- **Audience Restriction**: Tokens must match configured audience
- **Session Isolation**: Each request uses unique session identifiers

### MCP Integration Benefits

- **Real-time Data**: Access to live AWS pricing and documentation
- **Extensibility**: Easy to add new MCP servers for additional data sources
- **Standardization**: Uses open MCP protocol for interoperability
- **Isolation**: MCP servers run in separate processes for stability


## Next Steps

### Production Considerations

1. **Token Management**
   - Implement token refresh logic
   - Cache tokens until expiration
   - Handle token renewal gracefully

2. **Error Handling**
   - Implement retry logic with exponential backoff
   - Add circuit breaker patterns for MCP servers
   - Provide fallback responses for service failures

3. **Monitoring and Observability**
   - Set up CloudWatch metrics and alarms
   - Implement distributed tracing
   - Monitor authentication success/failure rates

4. **Scaling Considerations**
   - Configure appropriate memory and timeout settings
   - Consider connection pooling for MCP servers
   - Implement rate limiting if needed

### Additional MCP Servers

Explore additional MCP servers for enhanced functionality:
- **AWS CloudFormation MCP Server**: Template and stack management
- **AWS Cost Explorer MCP Server**: Detailed cost analysis
- **Custom MCP Servers**: Build domain-specific integrations


## Conclusion

This tutorial demonstrated how to:

✅ **Set up Okta OAuth 2.0** for secure authentication  
✅ **Deploy MCP Agent** on Bedrock Agent Core  
✅ **Integrate MCP servers** for real-time AWS data access  
✅ **Test the complete flow** with proper error handling  
✅ **Implement security best practices** for production use  

The MCP Agent architecture provides a robust foundation for building AI agents that can access real-time external data while maintaining strong security boundaries through OAuth authentication.

### Key Takeaways

- **OAuth Integration**: Seamless authentication with existing identity providers
- **MCP Protocol**: Standardized way to extend AI agents with external data
- **Bedrock Agent Core**: Managed runtime environment with built-in security
- **Scalable Architecture**: Production-ready patterns for enterprise deployment

For questions or issues, refer to the troubleshooting section or consult the [AWS Bedrock Agent Core documentation](https://docs.aws.amazon.com/bedrock/).
